In [1]:
import sys 
assert sys.version_info >= (3, 5)
import sklearn
assert sklearn.__version__ >= "0.20" 
try: 
  %tensorflow_version only exists in Colab.
except Exception: pass

import tensorflow as tf
assert tf.__version__ >= "2.0"

import numpy as np
import os

np.random.seed(42)

%matplotlib inline 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
mpl.rc('axes', labelsize=14) 
mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12)

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
np.random.seed(42) 
tf.random.set_seed(42)

input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu')(input_)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])
#2017250045 정태환
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [4]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test) 
#2017250045 정태환
X_new = X_test[:3] 
y_pred = model.predict(X_new)
print("y_pred",y_pred)

c:\ProgramData\Anaconda3\envs\aijth\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
363/363 [==============================] - 2s 2ms/step - loss: 1.3035 - val_loss: 0.7101
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6619 - val_loss: 0.6456
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.6156 - val_loss: 0.6133
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5873 - val_loss: 0.5928
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5664 - val_loss: 0.5708
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5470 - val_loss: 0.5567
Epoch 7/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5324 - val_loss: 0.5430
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5193 - val_loss: 0.5318
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5088 - val_loss: 0.5219
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4994 - val_loss: 0.5134

In [5]:
np.random.seed(42) 
tf.random.set_seed(42)

input_A = keras.layers.Input(shape=[5], name="wide_input") 
input_B = keras.layers.Input(shape=[6], name="deep_input") 
hidden1 = keras.layers.Dense(30, activation="relu")(input_B) 
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1) 
concat = keras.layers.concatenate([input_A, hidden2]) 
output = keras.layers.Dense(1, name="output")(concat) 
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss='mse', optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
#2017250045 정태환
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

print("mse_test:", mse_test)
print("y_pred",y_pred)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.8248 - val_loss: 0.7923
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.7055 - val_loss: 0.6450
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6258 - val_loss: 0.6057
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5910 - val_loss: 0.5840
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5690 - val_loss: 0.5665
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5508 - val_loss: 0.5542
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5373 - val_loss: 0.5430
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5256 - val_loss: 0.5343
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5168 - val_loss: 0.5270
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5093 - val_loss: 0.5214

In [6]:
model = keras.models.Sequential([ 
  keras.layers.Dense(30, activation="relu", input_shape=[8]), 
  keras.layers.Dense(30, activation="relu"),
  keras.layers.Dense(1) 
]) 
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid)) 
mse_test = model.evaluate(X_test, y_test)

model.save("my_keras_model.h5") 

model = keras.models.load_model("my_keras_model.h5")

model.predict(X_new) 
#2017250045 정태환
model.save_weights("my_keras_weights.ckpt") 

model.load_weights("my_keras_weights.ckpt")

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 1.8524 - val_loss: 0.9697
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.8411 - val_loss: 0.7685
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.7353 - val_loss: 0.7133
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.6879 - val_loss: 0.6739
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6500 - val_loss: 0.6403
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6172 - val_loss: 0.6125
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5888 - val_loss: 0.5883
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5643 - val_loss: 0.5665
Epoch 9/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5433 - val_loss: 0.5481
Epoch 10/10
1/1 [==============================] - 0s 44ms/step


In [7]:
keras.backend.clear_session()
np.random.seed(42) 
tf.random.set_seed(42) 

model = keras.models.Sequential([
  keras.layers.Dense(30, activation="relu", input_shape=[8]), 
  keras.layers.Dense(30, activation="relu"), 
  keras.layers.Dense(1) 
]) 
#2017250045 정태환
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3)) 
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True) 
history = model.fit(X_train, y_train, epochs=10,
  validation_data=(X_valid, y_valid), 
  callbacks=[checkpoint_cb]) 
model = keras.models.load_model("my_keras_model.h5") # rollback to best model 
mse_test = model.evaluate(X_test, y_test) 

model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3)) 
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, 
  restore_best_weights=True) 
history = model.fit(X_train, y_train, epochs=100,
  validation_data=(X_valid, y_valid), 
  callbacks=[checkpoint_cb, early_stopping_cb]) 
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 1.8688 - val_loss: 0.7796
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.7040 - val_loss: 0.6453
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6180 - val_loss: 0.6001
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5762 - val_loss: 0.5717
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5496 - val_loss: 0.5476
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.5267 - val_loss: 0.5303
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5100 - val_loss: 0.5157
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4960 - val_loss: 0.5045
Epoch 9/10
363/363 [==============================] - 1s 1ms/step - loss: 0.4856 - val_loss: 0.4955
Epoch 10/10
162/162 [==============================] - 0s 795us/step - loss: 0.4842
Epoch 1/100
363/

In [8]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    print('\nval/train: {:.2f}'.format(logs['val_loss'] / logs['loss']))
#2017250045 정태환
val_train_ratio_cb = PrintValTrainRatioCallback() 
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb])

323/363 [=========================>....] - ETA: 0s - loss: 0.3557
val/train: 1.07
363/363 [==============================] - 0s 1ms/step - loss: 0.3531 - val_loss: 0.3790


In [9]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir(): 
  import time 
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id) 
#2017250045 정태환
run_logdir = get_run_logdir() 
print("\nrun_logdir: \n", run_logdir)


run_logdir: 
 .\my_logs\run_2022_06_01-00_30_59


In [10]:
keras.backend.clear_session()

model = keras.models.Sequential([ 
  keras.layers.Dense(30, activation="relu", input_shape=[8]), 
  keras.layers.Dense(30, activation="relu"), 
  keras.layers.Dense(1) 
]) 
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
#2017250045 정태환
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
  validation_data=(X_valid, y_valid), 
  callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 1s 2ms/step - loss: 1.4905 - val_loss: 0.8458
Epoch 2/30
363/363 [==============================] - 0s 1ms/step - loss: 0.7623 - val_loss: 0.7172
Epoch 3/30
363/363 [==============================] - 0s 1ms/step - loss: 0.6803 - val_loss: 0.6600
Epoch 4/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6294 - val_loss: 0.6193
Epoch 5/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5930 - val_loss: 0.5879
Epoch 6/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5639 - val_loss: 0.5647
Epoch 7/30
363/363 [==============================] - 0s 1ms/step - loss: 0.5408 - val_loss: 0.5443
Epoch 8/30
363/363 [==============================] - 1s 1ms/step - loss: 0.5219 - val_loss: 0.5289
Epoch 9/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5065 - val_loss: 0.5162
Epoch 10/30
363/363 [==============================] - 0s 1ms/step - loss: 0.4943 - val_loss: 0.5059

In [11]:
keras.backend.clear_session()
np.random.seed(42) 
tf.random.set_seed(42)
#2017250045 정태환
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='relu'))
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss='mse', optimizer=optimizer)
  return model
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
model.evaluate(X_test, y_test)
print("model.evaluate(X_test, y_test):", model.evaluate(X_test, y_test))

mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)
print("\ny_pred: \n", y_pred)

Epoch 1/100


C:\Users\JTH\AppData\Local\Temp\ipykernel_12152\1004620348.py:14: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


363/363 [==============================] - 1s 1ms/step - loss: 1.0566 - val_loss: 0.7676
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6914 - val_loss: 0.6127
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5818 - val_loss: 0.5436
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5167 - val_loss: 0.5164
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4896 - val_loss: 0.4981
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4758 - val_loss: 0.4882
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4662 - val_loss: 0.4783
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4583 - val_loss: 0.4724
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4525 - val_loss: 0.4673
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4481 - val_loss: 0.4630
E

In [12]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': list(range(32,64)),
    'learning_rate': [3e-4, 3e-2]
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])
print("\nrnd_search_cv.best_params_: \n", rnd_search_cv.best_params_) 
print("\nrnd_search_cv.best_score_: \n", rnd_search_cv.best_score_)
print("\nrnd_search_cv.best_estimator_: \n", rnd_search_cv.best_estimator_) 
print("\nrnd_search_cv.score(X_test, y_test): \n", rnd_search_cv.score(X_test, y_test)) 
model = rnd_search_cv.best_estimator_.model 
print("\nmodel: \n", model) 

model.evaluate(X_test, y_test) 
print("\nmodel.evaluate(X_test, y_test): \n", model.evaluate(X_test, y_test))

Epoch 1/100


c:\ProgramData\Anaconda3\envs\aijth\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 0.6801 - val_loss: 0.4968
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4424 - val_loss: 0.5255
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4083 - val_loss: 0.4107
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3915 - val_loss: 0.3936
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3824 - val_loss: 0.4107
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3673 - val_loss: 0.3818
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3612 - val_loss: 0.3787
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3524 - val_loss: 0.4739
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3595 - val_loss: 0.3826
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3498 - val_loss: 0.3845
E

c:\ProgramData\Anaconda3\envs\aijth\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 0.7640 - val_loss: 0.5610
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 2.7364 - val_loss: 0.7631
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.6583 - val_loss: 0.4853
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4313 - val_loss: 0.4559
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3799 - val_loss: 0.4144
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3616 - val_loss: 0.3920
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3496 - val_loss: 0.3887
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3419 - val_loss: 0.3758
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3399 - val_loss: 0.3680
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3326 - val_loss: 0.3920
E

...

242/242 [==============================] - 1s 2ms/step - loss: 5.1278 - val_loss: 4.6669
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 3.9822 - val_loss: 3.6389
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 3.1354 - val_loss: 2.8819
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 2.5067 - val_loss: 2.3215
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 2.0388 - val_loss: 1.9059
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 1.6896 - val_loss: 1.5973
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 1.4288 - val_loss: 1.3675
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 1.2333 - val_loss: 1.1961
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0866 - val_loss: 1.0683
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.9764 - val_loss: 0.9726
E

c:\ProgramData\Anaconda3\envs\aijth\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 3ms/step - loss: 6.7378 - val_loss: 5.4453
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 5.0324 - val_loss: 4.1204
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 3.8267 - val_loss: 3.1771
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 2.9669 - val_loss: 2.4998
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 2.3487 - val_loss: 2.0104
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 1.9017 - val_loss: 1.6551
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 1.5765 - val_loss: 1.3960
Epoch 8/100
242/242 [==============================] - 0s 1ms/step - loss: 1.3386 - val_loss: 1.2058
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 1.1639 - val_loss: 1.0660
Epoch 10/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0349 - val_loss: 0.9630
E

c:\ProgramData\Anaconda3\envs\aijth\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [        nan -0.5334809  -0.45151356         nan -0.42591371         nan
         nan         nan -0.52686721 -0.54012064]
  warnings.warn(


363/363 [==============================] - 1s 2ms/step - loss: 3.6994 - val_loss: 2.0786
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 1.5724 - val_loss: 1.2896
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 1.0998 - val_loss: 0.9387
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.8536 - val_loss: 0.7656
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.7284 - val_loss: 0.6885
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6692 - val_loss: 0.6539
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6384 - val_loss: 0.6342
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6179 - val_loss: 0.6187
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.6018 - val_loss: 0.6053
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.5879 - val_loss: 0.5931
E